In [1]:
from external.coingecko.read_info_api import get_prices
from external.symbiotic.read_info_api import get_operators,get_networks,get_vaults
from external.symbiotic.read_info_cli import get_staking_data
from external.coingecko.read_info_api import get_prices
import polars as pl
from itertools import batched
import polars as pl
pl.Config.set_tbl_cols(50)
pl.Config.set_tbl_rows(20)  # Change 50 to any number

polars.config.Config

In [ ]:
### Get data from Symbiotic API
df_operators = await get_operators()
df_networks = await get_networks()
#df_vaults = await get_vaults()

df_staking_data = get_staking_data("python symb.py nets")

# Get unique addresses
unique_price_list = df_staking_data["collateral_asset_address"].unique().to_list()

# Store results in a list and convert to DataFrame at the end
price_results = []

for batch in batched(unique_price_list, 5):  # Batch size of 5
    contract_addresses_string = ",".join(batch)    
    price_results.append(await get_prices(contract_addresses_string))

# Convert the list of results into a Polars DataFrame
prices = pl.concat(price_results)

print(prices)

df_staking_data = df_staking_data.with_columns(
    pl.col("collateral_asset_address").lower().alias("collateral_asset_address")
)

df_staking_data = df_staking_data.join(
                        df_operators,
                        on=['node_operator_address'],
                        how='left'
                    ).join(
                        df_networks,
                        on=['network_address'],
                        how='left'
                        ).join(
                        prices,
                        on=['collateral_asset_address'],
                        how='left'
                    )

df_staking_data = df_staking_data.with_columns(
    pl.col("amount_staked") * pl.col("collateral_asset_price").alias("amount_staked_usd")
)
print(df_staking_data)
print(df_staking_data.null_count())

Getting operators from Symbiotic API
✅ Operators fetched successfully
Getting networks from Symbiotic API
✅ Networks fetched successfully
Running command: python symb.py nets
Output:
['Connected to chain ID 1', 'All networks [58 total]:', '  Network: 0xcffAca5f6887f307718ae41b9777ca509A5980F1', '    Middleware: 0x0000000000000000000000000000000000000000', '', '  Network: 0x9101eda106A443A0fA82375936D0D1680D5a64F5', '    Middleware: 0x21fD239311B050bbeE7F32850d99ADc224761382', '', '  Network: 0x759D4335cb712aa188935C2bD3Aa6D205aC61305', '    Middleware: 0x32710cF9ea90378c4fbCd5A0416D23633D00d5e8', '', '  Network: 0xE3a148b25Cca54ECCBD3A4aB01e235D154f03eFa', '    Middleware: 0xBf3e64f0f83d5ce7f9BfFA0eE7Ec524e9999D572', '', '  Network: 0x6c7C332a090c8D2085857cf3220eA01C6d45a723', '    Middleware: 0x0000000000000000000000000000000000000000', '', '  Network: 0x3934632DDE8224ae298e11CD040e732256a99D23', '    Middleware: 0x0000000000000000000000000000000000000000', '', '  Network: 0x97Ab4CcFb

In [ ]:
df_staking_data = df_staking_data.drop(["collateral_asset_price", "node_operator_name","network_name"])

In [4]:
df_staking_data

network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price
str,str,str,str,f64,datetime[μs],str,str,f64
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xf951E335afb289353dc249e82926…","""sweth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xBe9895146f7AF43049ca1c1AE358…","""cbeth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xa2E3356610840701BDf5611a5397…","""wbeth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xae78736Cd615f374D3085123A210…","""reth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x7f39C581F595B53c5cb19bD0b3f8…","""wsteth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x9D39A5DE30e57443BfF2A8307A42…","""susde""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0x6f64decffde23aa598724325202f…","""0x7f39C581F595B53c5cb19bD0b3f8…","""wsteth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0x9321d38c355d1d8cb9dbfc05a5c0…","""0x8c1BEd5b9a0928467c9B1341Da1D…","""lseth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xd741f826c60dacebb8835278e0f1…","""0xf951E335afb289353dc249e82926…","""sweth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null


In [6]:
df_operators

node_operator_address,node_operator_name
str,str
"""0x00e7ad2a172633037c16f4470040…","""BlockHunters 🎯"""
"""0x030d6216f27b4370d9e173ff6dd2…","""Node.Monster"""
"""0x0710613f6aa7623f47b8c1c3d036…","""Imperator.co"""
"""0x087c25f83ed20bda587cfa035ed0…","""P2P.org"""
"""0x09505f2b29062de4e91e0053490b…","""deNodes"""
"""0x09e6b2b6634c5b676198f32587de…","""Sitnoprodex"""
"""0x0e66cd3a8117b72f5bd8e81cc078…","""P-OPS Team"""
"""0x10ee0af996da1e206d96402b4f16…","""Nethermind"""
"""0x220b52b504d090c5aba1066bcf4b…","""Enigma"""


In [5]:
prices

collateral_asset_address,collateral_asset_price
str,f64
"""0x9d39a5de30e57443bff2a8307a42…",1.16
"""0xf1c9acdc66974dfb6decb12aa385…",1988.93
"""0xf951e335afb289353dc249e82926…",2070.97
"""0xd38bb40815d2b0c2d2c866e0c72c…",0.060421
"""0x20157dbabb84e3bbfe68c349d0d4…",83264.0
"""0xd5f7838f5c461feff7fe49ea5eba…",2025.89
"""0xc02aaa39b223fe8d0a0e5c4f27ea…",1909.49
"""0x2260fac5e5542a773aa44fbcfedf…",83546.0
"""0xfc385a1df85660a7e041423db512…",0.04611


In [17]:
df_staking_data.filter(pl.col("collateral_asset_price_right").is_null()).select(pl.col("collateral_asset_address")).row(0)[0]


'0x78c6b27be6db520d332b1b44323f94bc831f5e33'